In [1]:
#Importing Libraries
import numpy as np
import pandas as pd
import cv2
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

# Extracting face

In [2]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
print(face_cascade)
def face_extract(img):
    gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cropped=img[y:y+h, x:x+w]
    
    return cropped


<CascadeClassifier 0x7f2a25c8d2f0>


# Creating a database of 100 images per person

In [ ]:
cap=cv2.VideoCapture(0)
count=0
while(True):
    ret, frame=cap.read()
    if face_extract(frame) is not None:
        count+=1
        
        img= cv2.resize(face_extract(frame), (200,200))
        #img= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        file_name_path='./Faces/Kashish/'+str(count)+'.jpg'
        cv2.imwrite(file_name_path, img)
        
        cv2.putText(img, str(count), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,0),2)
        cv2.imshow('image',img)
        if cv2.waitKey(13)& 0xFF == ord('q'):
            break
        
    if count==100:
        break
        
cap.release()
cv2.destroyAllWindows()

# Data preparation

In [3]:
train=[]
label=[]
names=[ 'Kashish', 'Rakshit', 'Aditya']
for name in names:
    for i in range(1,100):       
        photo=cv2.imread('./Faces/'+name+'/'+ str(i)+'.jpg')
        train.append(np.asarray(photo))
        label.append(name)
 

In [4]:
data=pd.DataFrame({'input':train, 'label':label})
data['label']=label
df = data.sample(frac=1).reset_index(drop=True)

In [5]:
y=np.array(df['label'])
y.shape

(297,)

In [6]:
z=[]
for i in range(df.shape[0]):
    z.append(df['input'][i])
z=np.array(z)
z.shape

(297, 200, 200, 3)

In [7]:
X_train= z[:230]
X_train=X_train/255.0
X_test=z[230:]
X_test=X_test/255.0
y_train= y[:230]
y_test=y[230:]

In [8]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(230, 200, 200, 3)
(67, 200, 200, 3)
(230,)
(67,)


In [9]:
names=['Kashish', 'Aditya', 'Rakshit']
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(names)
print(integer_encoded)

[1 0 2]


In [10]:
y_tr=label_encoder.transform(y_train)
y_te=label_encoder.transform(y_test)
#print(y_tr)
#print(y_te)
print(y_test)

['Kashish' 'Aditya' 'Rakshit' 'Kashish' 'Aditya' 'Kashish' 'Aditya'
 'Aditya' 'Kashish' 'Aditya' 'Aditya' 'Aditya' 'Rakshit' 'Rakshit'
 'Aditya' 'Aditya' 'Aditya' 'Kashish' 'Rakshit' 'Rakshit' 'Rakshit'
 'Aditya' 'Rakshit' 'Aditya' 'Rakshit' 'Kashish' 'Aditya' 'Kashish'
 'Kashish' 'Aditya' 'Kashish' 'Aditya' 'Aditya' 'Kashish' 'Aditya'
 'Kashish' 'Rakshit' 'Rakshit' 'Rakshit' 'Rakshit' 'Aditya' 'Kashish'
 'Rakshit' 'Kashish' 'Aditya' 'Kashish' 'Rakshit' 'Rakshit' 'Rakshit'
 'Rakshit' 'Kashish' 'Aditya' 'Rakshit' 'Rakshit' 'Rakshit' 'Aditya'
 'Kashish' 'Rakshit' 'Kashish' 'Rakshit' 'Aditya' 'Aditya' 'Aditya'
 'Aditya' 'Kashish' 'Aditya' 'Rakshit']


# Model Building

In [3]:
import keras
import time

Using TensorFlow backend.


In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.callbacks import TensorBoard

In [13]:
from keras.utils import to_categorical
y_binary = to_categorical(y_tr)
y_binary_tst=to_categorical(y_te)
#print(y_binary)

In [15]:
model=Sequential()
name='cnn-{}'.format(int(time.time()))
model.add(Conv2D(64, (3,3), input_shape=(200,200,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))

model.add(Dense(3, activation='sigmoid'))

tensorboard = TensorBoard(log_dir="logs/{}".format(name))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

model.summary()

model.fit(X_train, y_binary, batch_size=32, epochs=5, validation_split=0.3, callbacks=[tensorboard])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 198, 198, 64)      1792      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 99, 99, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 97, 97, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 147456)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                9437248   
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 195       
Total para

In [16]:
val_loss, val_acc=model.evaluate(X_test, y_binary_tst)
print(val_loss, val_acc)

67/67 [==============================] - 1s 14ms/step
0.0014116546319825436 1.0


In [17]:
model.save('Face Recognition CNN')

In [4]:
cnn_model= keras.models.load_model('Face Recognition CNN')

W0629 11:42:36.266363 139819972069184 deprecation_wrapper.py:119] From /home/home/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0629 11:42:36.292382 139819972069184 deprecation_wrapper.py:119] From /home/home/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0629 11:42:36.316991 139819972069184 deprecation_wrapper.py:119] From /home/home/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0629 11:42:36.424427 139819972069184 deprecation_wrapper.py:119] From /home/home/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0629 11:4

In [5]:
def predict(x):
    ans=[]
    result=[]

    prediction=cnn_model.predict(x)
    prediction.shape
    
    for i in range(prediction.shape[0]):
        ans.append(np.argmax(prediction[i]))
        result=label_encoder.inverse_transform(ans)
    #print(ans)    
    #print(result)
    return result


In [14]:
print(predict(X_test))

['Kashish' 'Aditya' 'Rakshit' 'Kashish' 'Aditya' 'Kashish' 'Aditya'
 'Aditya' 'Kashish' 'Aditya' 'Aditya' 'Aditya' 'Rakshit' 'Rakshit'
 'Aditya' 'Aditya' 'Aditya' 'Kashish' 'Rakshit' 'Rakshit' 'Rakshit'
 'Aditya' 'Rakshit' 'Aditya' 'Rakshit' 'Kashish' 'Aditya' 'Kashish'
 'Kashish' 'Aditya' 'Kashish' 'Aditya' 'Aditya' 'Kashish' 'Aditya'
 'Kashish' 'Rakshit' 'Rakshit' 'Rakshit' 'Rakshit' 'Aditya' 'Kashish'
 'Rakshit' 'Kashish' 'Aditya' 'Kashish' 'Rakshit' 'Rakshit' 'Rakshit'
 'Rakshit' 'Kashish' 'Aditya' 'Rakshit' 'Rakshit' 'Rakshit' 'Aditya'
 'Kashish' 'Rakshit' 'Kashish' 'Rakshit' 'Aditya' 'Aditya' 'Aditya'
 'Aditya' 'Kashish' 'Aditya' 'Rakshit']


In [6]:
def face_extract(img):
    gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0),2)
        cropped=img[y:y+h, x:x+w]
        resized=cv2.resize(cropped, (200,200))
    return img,resized
    
    
cap=cv2.VideoCapture(0)
cnt=0
while(cnt<2):
    ret, frame=cap.read()
    
    img,face=face_extract(frame)
    result=predict(face)
    cv2.putText(img,result[0],(100,100), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0),2 )
    cv2.imshow('check', img)
    
    #print(face.shape)
    
    
    cnt+=1
    
    
   # try:
        #result=predict(face)
        #if result[1]<500:
         #   confidence_value = int(100* (1-(result[1]/300)))
        #if confidence_value>80:
         #   cv2.putText(img,"Confidence "+str(confidence_value),(100,100), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0),2 )
          #  cv2.imshow('check', img)
    #except:
        #cv2.putText(img,"NO FACE", (100,100),cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0),2 )
        #cv2.imshow('check', img)
        #pass
    if cv2.waitKey(13)& 0xFF == ord('q'):
        break
    if cnt==2:
        break
        
    
        
cap.release()
cv2.destroyAllWindows()

IndexError: list index out of range

In [ ]:
dense_layers=[0,1,2]
layer_sizes= [32, 64, 128]
conv_layers=[1,2,3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            name= "{}- conv-{}-nodes- {}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print("********************* "+name+" ************************")  
            
            model=Sequential()
            model.add(Conv2D(layer_size, (3,3), input_shape=X_train.shape[1:], activation='relu'))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for i in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3,3), activation='relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))
                
            model.add(Flatten())
            for i in range(dense_layer):
                model.add(Dense(layer_size, activation='relu'))
            
            model.add(Dense(3, activation='softmax')) 
            tensorboard = TensorBoard(log_dir="logs/{}".format(name))
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
            model.fit(X_train, y_binary, batch_size=32, epochs=10, validation_split=0.3, callbacks=[tensorboard])